# Here is the data cleaning of the dataset containing reviews

In [1]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Pablofuster/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('TA_restaurants_curated.csv')
df.head(5)

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959
3,3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",/Restaurant_Review-g188590-d1239229-Reviews-Vi...,d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",/Restaurant_Review-g188590-d6864170-Reviews-Li...,d6864170


In [3]:
df.shape

(125527, 11)

In [4]:
df.dtypes

Unnamed: 0             int64
Name                  object
City                  object
Cuisine Style         object
Ranking              float64
Rating               float64
Price Range           object
Number of Reviews    float64
Reviews               object
URL_TA                object
ID_TA                 object
dtype: object

In [5]:
#drop a column that does not interest me and set as index the unique one. 
df = df.drop(['Unnamed: 0','ID_TA'], axis=1)

In [6]:
#In order for the id to mathch with the one from the other dataset I clean it leaving only the code that I have on the other dataset
df['URL_TA'] = df['URL_TA'].str.replace("/Restaurant_Review-","")
df['URL_TA'] = df['URL_TA'].str.replace("(-Reviews).*","")

In [7]:
#Here we see the percentage of nulls in the columns of the dataset
nulls = pd.DataFrame(df.isna().sum()/len(df)*100, columns = ['percentage'])
nulls.sort_values('percentage',ascending=False)

,percentage
Price Range,38.123272
Cuisine Style,24.975503
Number of Reviews,13.816948
Ranking,7.688386
Rating,7.671656
Reviews,7.660503
Name,0.000000
City,0.000000
URL_TA,0.000000


In [8]:
#See the values of the column price and match them with the ones in the other dataset
df['Price Range'].value_counts()

$$ - $$$    54360
$           19005
$$$$         4307
Name: Price Range, dtype: int64

In [9]:
#Rename a few columns 
df = df.rename(columns={'Price Range': 'price_level', 'Cuisine Style': 'cuisine','Reviews':'reviews', 'URL_TA': 'id'})

In [10]:
#Change values in price from the dollas symbol to ranges of money
df['price_level'] = df['price_level'].replace('$$$$', '+45')
df['price_level'] = df['price_level'].fillna('Not Specified')
df['price_level'] = df['price_level'].replace('$', '0-20')
df['price_level'] = df['price_level'].replace('$$ - $$$', '20-45')

In [11]:
#Clean the cuisine columns in order to get only the first value
df['cuisine'] = df['cuisine'].str.replace("(,).*","").str.replace("'","").str.replace("]","").str.replace("[","").fillna('Not Specified')

In [12]:
#Use regex in order to remove all the number from that columns in order to clean it
df['reviews'] = df['reviews'].str.replace('\d+', '').fillna("Not Specified")

In [13]:
#df['Reviews'] = df['Reviews'].str.replace('[', '').str.replace("'","").str.replace(",","").str.replace("/","").str.replace("]","")

In [14]:
#Create a function to clean the text from the reviews
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [15]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
# Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])
df['reviews'] = df['reviews'].apply(text_process)

In [16]:
#Since the id in unique for each restaurant we put it at the beggining
first_column = df.pop('id')
df.insert(0, 'id', first_column)

In [17]:
#Fill the NaNs with the mean
df = df.fillna(df.mean())

In [18]:
df.isna().sum()

id                   0
Name                 0
City                 0
cuisine              0
Ranking              0
Rating               0
price_level          0
Number of Reviews    0
reviews              0
dtype: int64

In [19]:
df.head(1)

,id,Name,City,cuisine,Ranking,Rating,price_level,Number of Reviews,reviews
0,g188590-d11752080,Martine of Martine's Table,Amsterdam,French,1.0,5.0,20-45,136.0,like home Warm Welcome Wintry Amsterdam


In [20]:
#Found importing to SQL that we have duplicates we erase them 
df = df.drop_duplicates(subset=['id'])

In [21]:
df.shape

(125326, 9)

In [22]:
df.to_csv('reviews.csv', index=False)